# Exercise 4: File-Level Deduplication

## Learning Objectives

In this exercise, you will:
- Learn how to detect duplicate files by content hash
- Understand file-level vs record-level deduplication
- Calculate potential space savings
- See how hash-based file comparison works

## What is File-Level Deduplication?

**File-level deduplication** is the process of finding duplicate files based on their **content**, not their names or locations. 

### Key Concepts

1. **Content-Based Detection**: Two files are considered duplicates if they have the **same content**, even if they have different names or are stored in different locations.

2. **Hash Functions**: We use cryptographic hash functions (like MD5 or SHA-256) to create a unique "fingerprint" of each file's content. Files with the same hash have identical content.

3. **Space Savings**: By identifying duplicate files, you can:
   - Delete redundant copies
   - Create symbolic links instead of copies
   - Optimize storage systems
   - Reduce backup storage requirements

### File-Level vs Record-Level Deduplication

| Aspect | Record-Level | File-Level |
|--------|--------------|------------|
| **What it finds** | Duplicate rows/records in a dataset | Duplicate files in a storage system |
| **Comparison unit** | Individual data records | Entire files |
| **Use case** | Cleaning data tables, removing duplicate entries | Storage optimization, backup deduplication |
| **Example** | Two rows with same name/email in a CSV | Two files with identical content but different names |
| **Method** | Compare column values | Compare file content hashes |

### Real-World Examples

- **Backup Systems**: Many backup systems use file-level deduplication to avoid storing the same file multiple times
- **Cloud Storage**: Services like Dropbox, Google Drive use deduplication to save storage space
- **Version Control**: Git uses content-based addressing (similar concept) to store files efficiently
- **Data Lakes**: Large data storage systems deduplicate files to optimize storage costs

### How It Works

1. **Read each file** and compute its hash (MD5, SHA-256, etc.)
2. **Compare hashes** - files with identical hashes have identical content
3. **Group duplicates** - identify which files are duplicates of each other
4. **Calculate savings** - determine how much space could be saved by removing duplicates

### Why Use Hash Functions?

- **Fast comparison**: Comparing two hashes is much faster than comparing entire file contents
- **Deterministic**: Same content always produces the same hash
- **Collision-resistant**: Different content almost never produces the same hash (for good hash functions)
- **Fixed size**: Hashes are always the same size regardless of file size (e.g., MD5 = 32 hex chars, SHA-256 = 64 hex chars)

In [ ]:
# Setup: Add project root to Python path
import sys
import os

# Find project root
current_dir = os.getcwd()
if 'notebooks' in current_dir:
    project_root = os.path.dirname(current_dir)
elif os.path.exists(os.path.join(current_dir, 'deduplicate_spark.py')):
    project_root = current_dir
else:
    # Search up directories
    test_dir = current_dir
    for _ in range(5):
        if os.path.exists(os.path.join(test_dir, 'deduplicate_spark.py')):
            project_root = test_dir
            break
        parent = os.path.dirname(test_dir)
        if parent == test_dir:
            break
        test_dir = parent
    project_root = project_root or current_dir

if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"✓ Added to Python path: {project_root}")

# Change to project root for file operations
os.chdir(project_root)
print(f"✓ Changed working directory to: {project_root}")


In [ ]:
from deduplicate_spark import create_spark_session, deduplicate_files
import os
import glob
import subprocess

spark = create_spark_session("Exercise4_FileDeduplication")
print("✓ Spark session created")

## Step 1: Generate or Find Duplicate Files

We'll create a set of test files, some of which will be duplicates. The `generate_duplicate_files.py` script creates files with intentional duplicates based on a duplication rate.


## Step 2: Run File-Level Deduplication

Now we'll use Spark to:
1. Compute a hash (MD5) for each file's content
2. Group files by their hash to find duplicates
3. Identify which files are unique and which are duplicates
4. Calculate potential space savings

The `deduplicate_files()` function handles all of this automatically.


In [ ]:
duplicate_files_dir = os.path.join(project_root, "data", "duplicatefiles")
data_dir = os.path.join(project_root, "data")

if not os.path.exists(duplicate_files_dir) or len(glob.glob(os.path.join(duplicate_files_dir, "*"))) == 0:
    print("Generating duplicate files...")
    # Create data directory if needed
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        print(f"✓ Created data directory: {data_dir}")
    
    script_path = os.path.join(project_root, "generate_duplicate_files.py")
    result = subprocess.run(
        ["python", script_path, "25", "0.9", duplicate_files_dir],
        cwd=project_root,
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        print("✓ Files generated")
    else:
        print(f"✗ Error generating files: {result.stderr}")
else:
    print("✓ Using existing files")

# Get all files
file_paths = glob.glob(os.path.join(duplicate_files_dir, "*"))
file_paths = [f for f in file_paths if os.path.isfile(f)]
print(f"\nFound {len(file_paths)} files to analyze")

if not os.path.exists(duplicate_files_dir) or len(glob.glob(os.path.join(duplicate_files_dir, "*"))) == 0:
    print("Generating duplicate files...")
    # Create data directory if needed
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
        print(f"✓ Created data directory: {data_dir}")
    
    script_path = os.path.join(project_root, "generate_duplicate_files.py")
    result = subprocess.run(
        ["python", script_path, "25", "0.9", duplicate_files_dir],
        cwd=project_root,
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        print("✓ Files generated")
    else:
        print(f"✗ Error generating files: {result.stderr}")
else:
    print("✓ Using existing files")

# Get all files
file_paths = glob.glob(os.path.join(duplicate_files_dir, "*"))
file_paths = [f for f in file_paths if os.path.isfile(f)]
print(f"\nFound {len(file_paths)} files to analyze")

In [ ]:
print("Running file-level deduplication...")
deduplicate_files(spark, file_paths, output_dir=None)
print("\n✓ Deduplication complete!")

## Understanding the Results

After running the deduplication, you should see:
- **Total files analyzed**: How many files were checked
- **Unique files**: Files with unique content (different hashes)
- **Duplicate groups**: Groups of files that share the same content
- **Space savings**: How much storage space could be freed by removing duplicates

### How to Interpret the Results

1. **If two files have the same hash**: They have identical content, even if they have different names
2. **Space savings calculation**: If 5 files share the same content, you could keep 1 and delete 4, saving 80% of the space used by those files
3. **Hash collisions**: With MD5 or SHA-256, collisions (different content, same hash) are extremely rare in practice

## Questions to Answer

1. **How many duplicate files were found?** Look at the duplicate groups count
2. **What is the total space that could be saved?** Check the space savings calculation
3. **How does file deduplication differ from record deduplication?** 
   - File-level: Compares entire files using content hashes
   - Record-level: Compares individual rows/records in a dataset
4. **Why use hash functions instead of comparing files byte-by-byte?**
   - Much faster: Hash comparison is O(1) vs O(n) for byte comparison
   - Fixed size: Hashes are always the same size regardless of file size
   - Efficient: Can quickly identify potential duplicates before detailed comparison
5. **What would happen if you had 1000 files, and 100 of them were duplicates of 10 unique files?**
   - You'd have 10 unique files
   - 90 duplicate files that could be removed
   - Significant space savings!

## Key Takeaways

- **File-level deduplication** finds duplicate files by comparing content hashes
- **Hash functions** create unique fingerprints of file content
- **Space savings** can be significant when many duplicate files exist
- **Use cases**: Backup systems, cloud storage, data lakes, storage optimization
- **Trade-off**: Hash computation takes time, but enables fast duplicate detection

In [ ]:
spark.stop()
print("✓ Spark session stopped")